## Importing Libraries

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test_NSBVHN8_tXljBsH_vH3Jate.csv")
campaign_data=pd.read_csv("campaign_data.csv")
coupon_item_mapping=pd.read_csv("coupon_item_mapping.csv")
customer_demographics=pd.read_csv("customer_demographics.csv")
customer_transaction_data=pd.read_csv("customer_transaction_data.csv")
item_data=pd.read_csv("item_data.csv")
submission=pd.read_csv("sample_submission_Byiv0dS_UQXM1Xo.csv")

In [7]:
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier,Pool, cv
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from sklearn.metrics import roc_auc_score

In [8]:
train.head()

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0
3,7,13,644,1050,0
4,9,8,1017,1489,0


In [9]:
train.shape

(78369, 5)

In [10]:
train.redemption_status.value_counts()

0    77640
1      729
Name: redemption_status, dtype: int64

In [11]:
campaign_data.shape

(28, 4)

In [12]:
campaign_data.head()

,campaign_id,campaign_type,start_date,end_date
0,24,Y,21/10/13,20/12/13
1,25,Y,21/10/13,22/11/13
2,20,Y,07/09/13,16/11/13
3,23,Y,08/10/13,15/11/13
4,21,Y,16/09/13,18/10/13


## Training and Testing

In [13]:
df=train.append(test,ignore_index=True)
df.head()

C:\Users\DEEKSHA\AppData\Local\Temp\ipykernel_21776\4069459972.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=train.append(test,ignore_index=True)


,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0.0
1,2,13,116,48,0.0
2,6,9,635,205,0.0
3,7,13,644,1050,0.0
4,9,8,1017,1489,0.0


## Feature Engineering

In [14]:
campaign_data['start_date']=pd.to_datetime(campaign_data['start_date'],format='%d/%m/%y',dayfirst=True)
campaign_data['end_date']=pd.to_datetime(campaign_data['end_date'],format='%d/%m/%y',dayfirst=True)

campaign_data['diff_d']=(campaign_data['end_date']-campaign_data['start_date'])/np.timedelta64(1,'D')
campaign_data['diff_m']=(campaign_data['end_date']-campaign_data['start_date'])/np.timedelta64(1,'M')
campaign_data['diff_w']=(campaign_data['end_date']-campaign_data['start_date'])/np.timedelta64(1,'W')

In [15]:
campaign_data.head()

,campaign_id,campaign_type,start_date,end_date,diff_d,diff_m,diff_w
0,24,Y,2013-10-21,2013-12-20,60.0,1.971293,8.571429
1,25,Y,2013-10-21,2013-11-22,32.0,1.051356,4.571429
2,20,Y,2013-09-07,2013-11-16,70.0,2.299842,10.000000
3,23,Y,2013-10-08,2013-11-15,38.0,1.248486,5.428571
4,21,Y,2013-09-16,2013-10-18,32.0,1.051356,4.571429


In [16]:
df=df.merge(campaign_data,on='campaign_id',how='left')
df.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,diff_d,diff_m,diff_w
0,1,13,27,1053,0.0,X,2013-05-19,2013-07-05,47.0,1.544180,6.714286
1,2,13,116,48,0.0,X,2013-05-19,2013-07-05,47.0,1.544180,6.714286
2,6,9,635,205,0.0,Y,2013-03-11,2013-04-12,32.0,1.051356,4.571429
3,7,13,644,1050,0.0,X,2013-05-19,2013-07-05,47.0,1.544180,6.714286
4,9,8,1017,1489,0.0,X,2013-02-16,2013-04-05,48.0,1.577034,6.857143


## Item Data and Coupon Item Mapping

In [17]:
for i in ['brand', 'brand_type', 'category']:
    print(i,item_data[i].nunique())

brand 5528
brand_type 2
category 19


In [18]:
for i in ['brand', 'brand_type', 'category']:
    item_data[i]=item_data[i].astype('category')
coupon_item_mapping=coupon_item_mapping.merge(item_data,on='item_id',how='left')

In [19]:
coupon_item_mapping.coupon_id.nunique()

1116

In [20]:
coupon_item_mapping.head()

,coupon_id,item_id,brand,brand_type,category
0,105,37,56,Local,Grocery
1,107,75,56,Local,Grocery
2,494,76,209,Established,Grocery
3,522,77,278,Established,Grocery
4,518,77,278,Established,Grocery


In [21]:
coupon_item_mapping.shape

(92663, 5)

## Feature Engineering

In [22]:
customer_transaction_data=pd.read_csv('customer_transaction_data.csv')
customer_transaction_data['date']=pd.to_datetime(customer_transaction_data['date'],format='%Y-%m-%d')
customer_transaction_data['date_d']=customer_transaction_data['date'].dt.day.astype('category')
customer_transaction_data['date_m']=customer_transaction_data['date'].dt.month.astype('category')
customer_transaction_data['date_w']=customer_transaction_data['date'].dt.week.astype('category')

C:\Users\DEEKSHA\AppData\Local\Temp\ipykernel_21776\890528401.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  customer_transaction_data['date_w']=customer_transaction_data['date'].dt.week.astype('category')


In [23]:
customer_transaction_data.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,date_d,date_m,date_w
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,2,1,1
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0,2,1,1
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0,2,1,1
3,2012-01-02,1501,33647,1,67.32,0.00,0.0,2,1,1
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0,2,1,1


In [24]:
customer_transaction_data[customer_transaction_data['quantity']==10]

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,date_d,date_m,date_w
312,2012-01-04,188,56795,10,245.78,0.00,0.0,4,1,1
505,2012-01-04,713,7364,10,213.72,-32.06,0.0,4,1,1
512,2012-01-04,713,21960,10,71.24,-17.45,0.0,4,1,1
1546,2012-01-09,368,32898,10,178.10,0.00,0.0,9,1,2
2640,2012-01-12,1568,19539,10,356.20,-530.74,0.0,12,1,2
...,...,...,...,...,...,...,...,...,...,...
1318444,2013-07-01,805,36797,10,356.20,-352.64,0.0,1,7,27
1319467,2013-07-01,474,13700,10,445.25,-263.59,0.0,1,7,27
1322055,2013-07-02,1552,11513,10,356.20,-495.12,0.0,2,7,27
1323738,2013-07-03,1478,46358,10,142.48,0.00,0.0,3,7,27


In [25]:
customer_transaction_data=customer_transaction_data.merge(coupon_item_mapping,on='item_id',how='left')
customer_transaction_data.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,date_d,date_m,date_w,coupon_id,brand,brand_type,category
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,2,1,1,7.0,56,Local,Natural Products
1,2012-01-02,1501,26830,1,35.26,-10.69,0.0,2,1,1,20.0,56,Local,Natural Products
2,2012-01-02,1501,26830,1,35.26,-10.69,0.0,2,1,1,29.0,56,Local,Natural Products
3,2012-01-02,1501,54253,1,53.43,-13.89,0.0,2,1,1,7.0,56,Local,Natural Products
4,2012-01-02,1501,54253,1,53.43,-13.89,0.0,2,1,1,20.0,56,Local,Natural Products


In [26]:
customer_transaction_data.shape

(2657495, 14)

In [27]:
customer_transaction_data=customer_transaction_data[customer_transaction_data.duplicated()==False]
customer_transaction_data.shape,train.shape

((2650887, 14), (78369, 5))

In [28]:
customer_transaction_data.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,date_d,date_m,date_w,coupon_id,brand,brand_type,category
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,2,1,1,7.0,56,Local,Natural Products
1,2012-01-02,1501,26830,1,35.26,-10.69,0.0,2,1,1,20.0,56,Local,Natural Products
2,2012-01-02,1501,26830,1,35.26,-10.69,0.0,2,1,1,29.0,56,Local,Natural Products
3,2012-01-02,1501,54253,1,53.43,-13.89,0.0,2,1,1,7.0,56,Local,Natural Products
4,2012-01-02,1501,54253,1,53.43,-13.89,0.0,2,1,1,20.0,56,Local,Natural Products


In [29]:
customer_transaction_data[(customer_transaction_data['customer_id']==1501) & (customer_transaction_data['coupon_id']==21)]

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,date_d,date_m,date_w,coupon_id,brand,brand_type,category
5250,2012-01-13,1501,31248,2,153.88,-73.38,0.00,13,1,2,21.0,461,Established,Grocery
927388,2012-08-30,1501,34047,1,82.64,0.00,0.00,30,8,35,21.0,56,Local,Grocery
1357079,2012-11-13,1501,21340,1,45.95,0.00,0.00,13,11,46,21.0,56,Local,Grocery
1357091,2012-11-13,1501,35284,3,106.86,-20.30,0.00,13,11,46,21.0,56,Local,Grocery
1362068,2012-11-13,1501,14633,1,63.40,0.00,0.00,13,11,46,21.0,56,Local,Grocery
1362070,2012-11-13,1501,21813,1,63.40,0.00,0.00,13,11,46,21.0,56,Local,Grocery
1362077,2012-11-13,1501,22985,2,241.50,-21.37,0.00,13,11,46,21.0,1075,Established,Grocery
1362082,2012-11-13,1501,31248,1,75.87,0.00,0.00,13,11,46,21.0,461,Established,Grocery
1431710,2012-11-24,1501,34047,1,89.76,0.00,0.00,24,11,47,21.0,56,Local,Grocery
1450629,2012-11-28,1501,13835,1,63.76,0.00,0.00,28,11,48,21.0,56,Local,Grocery


In [30]:
df[(df['customer_id']==1501) & (df['coupon_id']==21)]

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,diff_d,diff_m,diff_w
41509,68124,13,21,1501,1.0,X,2013-05-19,2013-07-05,47.0,1.54418,6.714286


In [31]:
def func(a,b,c):
    if c!=0:
        c=list(c)
        x=0
        for i in c:
            if a<=i and b>i:
                x+=1
        return x
    else:
        return 0

In [32]:
customer_transaction_data.columns

Index(['date', 'customer_id', 'item_id', 'quantity', 'selling_price',
       'other_discount', 'coupon_discount', 'date_d', 'date_m', 'date_w',
       'coupon_id', 'brand', 'brand_type', 'category'],
      dtype='object')

In [33]:
ddf = df.merge(customer_transaction_data.groupby(['customer_id', 'coupon_id']).agg({'date': set,'quantity': 'sum','item_id': 'count'}).reset_index(),on=['customer_id', 'coupon_id'],how='left')
ddf.sample(10)

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,diff_d,diff_m,diff_w,date,quantity,item_id
29100,47697,13,987,441,0.0,X,2013-05-19,2013-07-05,47.0,1.544180,6.714286,NaN,NaN,NaN
108853,78228,17,583,1431,NaN,Y,2013-07-29,2013-08-30,32.0,1.051356,4.571429,NaN,NaN,NaN
22067,36185,30,814,580,0.0,X,2012-11-19,2013-01-04,46.0,1.511325,6.571429,NaN,NaN,NaN
29491,48314,13,467,1216,0.0,X,2013-05-19,2013-07-05,47.0,1.544180,6.714286,NaN,NaN,NaN
91576,33712,18,1024,655,NaN,X,2013-08-10,2013-10-04,55.0,1.807019,7.857143,{2012-09-16 00:00:00},1.0,1.0
51577,84648,10,15,515,0.0,Y,2013-04-08,2013-05-10,32.0,1.051356,4.571429,NaN,NaN,NaN
92114,35182,20,836,1464,NaN,Y,2013-09-07,2013-11-16,70.0,2.299842,10.000000,NaN,NaN,NaN
60490,99372,13,119,1255,0.0,X,2013-05-19,2013-07-05,47.0,1.544180,6.714286,NaN,NaN,NaN
36911,60569,9,691,902,0.0,Y,2013-03-11,2013-04-12,32.0,1.051356,4.571429,NaN,NaN,NaN
13522,22322,8,65,684,0.0,X,2013-02-16,2013-04-05,48.0,1.577034,6.857143,NaN,NaN,NaN


In [34]:
ddf['coupon_id'].fillna(0)

0           27
1          116
2          635
3          644
4         1017
          ... 
128590     843
128591     415
128592     596
128593     518
128594     988
Name: coupon_id, Length: 128595, dtype: int64

In [35]:
ddf.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,diff_d,diff_m,diff_w,date,quantity,item_id
0,1,13,27,1053,0.0,X,2013-05-19,2013-07-05,47.0,1.544180,6.714286,NaN,NaN,NaN
1,2,13,116,48,0.0,X,2013-05-19,2013-07-05,47.0,1.544180,6.714286,NaN,NaN,NaN
2,6,9,635,205,0.0,Y,2013-03-11,2013-04-12,32.0,1.051356,4.571429,NaN,NaN,NaN
3,7,13,644,1050,0.0,X,2013-05-19,2013-07-05,47.0,1.544180,6.714286,NaN,NaN,NaN
4,9,8,1017,1489,0.0,X,2013-02-16,2013-04-05,48.0,1.577034,6.857143,NaN,NaN,NaN


In [36]:
c = ['count', 'nunique']
n = ['mean', 'max', 'min', 'sum', 'std']
agg_c = {'date_d': c,'date_m': c,'date_w': c,'quantity': n,'selling_price': n,'other_discount': n,'coupon_discount': n,'item_id': c,'coupon_id': c}
trans = customer_transaction_data.groupby('customer_id').agg(agg_c)
trans.head()

date_d         date_m         date_w           quantity         \
             count nunique  count nunique  count nunique       mean    max   
customer_id                                                                  
1             2206      28   2206      12   2206      38   1.133726      5   
2              823      19    823      11    823      27   1.117861      5   
3             1351      23   1351      12   1351      30   6.861584   6949   
4              451      23    451      11    451      21   1.283814      5   
5             1594      31   1594      12   1594      51  59.345671  14638   

                        ... other_discount coupon_discount               \
            min    sum  ...            std            mean  max     min   
customer_id             ...                                               
1             1   2501  ...      36.111393       -1.864565  0.0 -106.86   
2             1    920  ...      33.011891       -0.302965  0.0  -89.05   
3             1   9270  ...      40.760823       -2.578209  0.0 -142.48   
4             1    579  ...      31.132399       -0.197450  0.0  -89.05   
5             1  94597  ...      28.304603       -0.126255  0.0  -71.24   

                                item_id         coupon_id          
                 sum        std   count nunique     count nunique  
customer_id                                                        
1           -4113.23   8.521853    2206     463      1665     159  
2            -249.34   4.721158     823     352       672     137  
3           -3483.16  11.568789    1351     406      1040     135  
4             -89.05   4.193201     451     125       360      66  
5            -201.25   2.660559    1594     490      1301     121  

[5 rows x 30 columns]

In [37]:
trans.columns=['F_' + '_'.join(col).strip() for col in trans.columns.to_flat_index()]
trans.reset_index(inplace=True)

In [38]:
trans.head()

,customer_id,F_date_d_count,F_date_d_nunique,F_date_m_count,F_date_m_nunique,F_date_w_count,F_date_w_nunique,F_quantity_mean,F_quantity_max,F_quantity_min,...,F_other_discount_std,F_coupon_discount_mean,F_coupon_discount_max,F_coupon_discount_min,F_coupon_discount_sum,F_coupon_discount_std,F_item_id_count,F_item_id_nunique,F_coupon_id_count,F_coupon_id_nunique
0,1,2206,28,2206,12,2206,38,1.133726,5,1,...,36.111393,-1.864565,0.0,-106.86,-4113.23,8.521853,2206,463,1665,159
1,2,823,19,823,11,823,27,1.117861,5,1,...,33.011891,-0.302965,0.0,-89.05,-249.34,4.721158,823,352,672,137
2,3,1351,23,1351,12,1351,30,6.861584,6949,1,...,40.760823,-2.578209,0.0,-142.48,-3483.16,11.568789,1351,406,1040,135
3,4,451,23,451,11,451,21,1.283814,5,1,...,31.132399,-0.197450,0.0,-89.05,-89.05,4.193201,451,125,360,66
4,5,1594,31,1594,12,1594,51,59.345671,14638,1,...,28.304603,-0.126255,0.0,-71.24,-201.25,2.660559,1594,490,1301,121


In [39]:
trans.shape

(1582, 31)

In [40]:
df=df.merge(trans,on=['customer_id'],how='left')

In [41]:
df['campaign_type']=df['campaign_type'].astype('category')

In [42]:
df_train=df[df['redemption_status'].isnull()==False].copy()
df_train.shape

(78369, 41)

In [43]:
df_test=df[df['redemption_status'].isnull()==True].copy()
df_test.shape

(50226, 41)

In [44]:
df_train.merge(df_train.drop(['id','redemption_status'],axis=1).groupby('campaign_id').mean().reset_index(),on='campaign_id',how='left')

C:\Users\DEEKSHA\AppData\Local\Temp\ipykernel_21776\1732192551.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_train.merge(df_train.drop(['id','redemption_status'],axis=1).groupby('campaign_id').mean().reset_index(),on='campaign_id',how='left')


,id,campaign_id,coupon_id_x,customer_id_x,redemption_status,campaign_type,start_date,end_date,diff_d_x,diff_m_x,...,F_other_discount_std_y,F_coupon_discount_mean_y,F_coupon_discount_max_y,F_coupon_discount_min_y,F_coupon_discount_sum_y,F_coupon_discount_std_y,F_item_id_count_y,F_item_id_nunique_y,F_coupon_id_count_y,F_coupon_id_nunique_y
0,1,13,27,1053,0.0,X,2013-05-19,2013-07-05,47.0,1.544180,...,36.212801,-0.576789,0.0,-84.322937,-1238.194375,4.316949,2008.680439,579.598071,1583.989383,159.237769
1,2,13,116,48,0.0,X,2013-05-19,2013-07-05,47.0,1.544180,...,36.212801,-0.576789,0.0,-84.322937,-1238.194375,4.316949,2008.680439,579.598071,1583.989383,159.237769
2,6,9,635,205,0.0,Y,2013-03-11,2013-04-12,32.0,1.051356,...,35.796547,-0.813429,0.0,-83.575000,-1535.185682,4.775683,1928.704545,541.607955,1549.352273,157.960227
3,7,13,644,1050,0.0,X,2013-05-19,2013-07-05,47.0,1.544180,...,36.212801,-0.576789,0.0,-84.322937,-1238.194375,4.316949,2008.680439,579.598071,1583.989383,159.237769
4,9,8,1017,1489,0.0,X,2013-02-16,2013-04-05,48.0,1.577034,...,37.060466,-0.543798,0.0,-82.199367,-1141.176194,4.219185,1891.250277,551.425538,1494.092662,153.839636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78364,128587,8,71,1523,0.0,X,2013-02-16,2013-04-05,48.0,1.577034,...,37.060466,-0.543798,0.0,-82.199367,-1141.176194,4.219185,1891.250277,551.425538,1494.092662,153.839636
78365,128589,30,547,937,0.0,X,2012-11-19,2013-01-04,46.0,1.511325,...,38.016761,-0.637498,0.0,-100.772994,-1486.139188,4.980882,2005.261017,574.846142,1596.233338,157.536725
78366,128590,8,754,1004,0.0,X,2013-02-16,2013-04-05,48.0,1.577034,...,37.060466,-0.543798,0.0,-82.199367,-1141.176194,4.219185,1891.250277,551.425538,1494.092662,153.839636
78367,128592,13,134,71,0.0,X,2013-05-19,2013-07-05,47.0,1.544180,...,36.212801,-0.576789,0.0,-84.322937,-1238.194375,4.316949,2008.680439,579.598071,1583.989383,159.237769


In [45]:
df_train=df_train.merge(df_train.drop(['id','redemption_status'], axis=1).groupby('coupon_id').mean(numeric_only=True).reset_index(),on='coupon_id',how='left')
df_test=df_test.merge(df_test.drop(['id','redemption_status'], axis=1).groupby('coupon_id').mean(numeric_only=True).reset_index(), on='coupon_id',how='left')

In [46]:
df_train[df_train.redemption_status==1]

,id,campaign_id_x,coupon_id,customer_id_x,redemption_status,campaign_type,start_date,end_date,diff_d_x,diff_m_x,...,F_other_discount_std_y,F_coupon_discount_mean_y,F_coupon_discount_max_y,F_coupon_discount_min_y,F_coupon_discount_sum_y,F_coupon_discount_std_y,F_item_id_count_y,F_item_id_nunique_y,F_coupon_id_count_y,F_coupon_id_nunique_y
44,72,13,413,276,1.0,X,2013-05-19,2013-07-05,47.0,1.544180,...,35.927525,-0.533482,0.0,-91.992200,-1259.605600,4.499910,2334.480000,653.220000,1847.130000,172.190000
181,299,13,960,467,1.0,X,2013-05-19,2013-07-05,47.0,1.544180,...,36.972150,-0.763208,0.0,-95.662791,-1712.240698,4.931871,2056.062016,615.558140,1618.844961,170.209302
217,359,26,797,214,1.0,X,2012-08-12,2012-09-21,40.0,1.314195,...,39.845991,-0.862074,0.0,-108.824516,-2630.590323,5.867636,2631.032258,713.129032,2089.451613,186.032258
237,387,13,754,1103,1.0,X,2013-05-19,2013-07-05,47.0,1.544180,...,36.233271,-0.647742,0.0,-83.023576,-1368.479503,4.440507,2134.201987,612.407285,1695.728477,168.112583
607,998,13,21,1457,1.0,X,2013-05-19,2013-07-05,47.0,1.544180,...,39.625657,-0.627733,0.0,-91.409412,-1315.472206,4.667072,2004.242647,571.014706,1594.941176,157.352941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77617,127381,13,982,1408,1.0,X,2013-05-19,2013-07-05,47.0,1.544180,...,35.632087,-0.602073,0.0,-81.589824,-1144.575991,4.368733,1991.731278,556.889868,1583.431718,153.625551
77729,127565,13,885,781,1.0,X,2013-05-19,2013-07-05,47.0,1.544180,...,36.598147,-0.570931,0.0,-81.907925,-1202.183776,4.153279,1937.721992,551.008299,1520.651452,153.174274
77830,127701,13,726,407,1.0,X,2013-05-19,2013-07-05,47.0,1.544180,...,37.390934,-0.652334,0.0,-91.732933,-1509.812200,4.784796,2126.900000,618.926667,1692.173333,168.020000
78060,128076,9,705,999,1.0,Y,2013-03-11,2013-04-12,32.0,1.051356,...,36.886975,-0.710382,0.0,-91.073000,-1540.043754,4.777980,2134.715789,602.517544,1699.771930,166.052632


In [47]:
loc=['id','redemption_status','start_date','end_date']
X,y=df_train.drop(loc,axis=1),df_train['redemption_status']
Xtest=df_test.drop(loc,axis=1)

In [48]:
X.shape

(78369, 72)

In [49]:
Xtest.shape

(50226, 72)

In [50]:
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.3,random_state = 1994,stratify=y)

In [51]:
col=['campaign_id', 'coupon_id', 'customer_id', 'campaign_type','within_date', 'within_date_discount']

### LGBM Model

In [52]:
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from sklearn.metrics import roc_auc_score
m=LGBMClassifier(n_estimators=1500,random_state=1994,learning_rate=0.03,reg_alpha=0.2,colsample_bytree=0.5,bagging_fraction=0.9)
m.fit(X_train,y_train,eval_set=[(X_train, y_train), (X_val, y_val)],eval_metric='auc',callbacks=[early_stopping(stopping_rounds=100),log_evaluation(period=200)])
p = m.predict_proba(X_val)[:, -1]
print(roc_auc_score(y_val, p))

[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Info] Number of positive: 510, number of negative: 54348
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030470 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13905
[LightGBM] [Info] Number of data points in the train set: 54858, number of used features: 66
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.009297 -> initscore=-4.668752
[LightGBM] [Info] Start training from score -4.668752
Training until validation scores don't improve for 100 rounds
[200]	training's auc: 0.998564	training's binary_logloss: 0.0153597	valid_1's auc: 0.932482	va

In [59]:
m.feature_importances_

array([ 61, 390, 185,  34,  41,  22,   8,  89,  60,  60,   5,  39, 109,
       102, 104,   0,  98,  79, 185, 117,  85, 139, 126, 122,   0, 166,
       139, 187, 246,   0, 144, 262, 187,  12, 128,  79, 166,  72, 163,
        51,  38,  13,  59,  97,  31,  89,  22, 113,  83, 144,   0,  81,
       116, 108,  95,  94,  72, 133, 133,   0, 192,  88, 130, 164,   0,
       103, 114, 102,   3,  67,  57,  97])

In [62]:
confusion_matrix(y_val,p>0.5)

ValueError: Found input variables with inconsistent numbers of samples: [15673, 23511]

In [54]:
confusion_matrix(y_val,p>0.5)

array([[23277,    15],
       [  208,    11]], dtype=int64)

In [61]:
sorted(zip(m.feature_importances_,X_train),reverse=True)

[(390, 'coupon_id'),
 (262, 'F_coupon_discount_sum_x'),
 (246, 'F_coupon_discount_mean_x'),
 (192, 'F_other_discount_min_y'),
 (187, 'F_other_discount_std_x'),
 (187, 'F_coupon_discount_std_x'),
 (185, 'customer_id_x'),
 (185, 'F_selling_price_mean_x'),
 (166, 'F_other_discount_min_x'),
 (166, 'F_coupon_id_nunique_x'),
 (164, 'F_coupon_discount_mean_y'),
 (163, 'customer_id_y'),
 (144, 'F_quantity_max_y'),
 (144, 'F_coupon_discount_min_x'),
 (139, 'F_selling_price_sum_x'),
 (139, 'F_other_discount_sum_x'),
 (133, 'F_selling_price_std_y'),
 (133, 'F_other_discount_mean_y'),
 (130, 'F_other_discount_std_y'),
 (128, 'F_item_id_nunique_x'),
 (126, 'F_selling_price_std_x'),
 (122, 'F_other_discount_mean_x'),
 (117, 'F_selling_price_max_x'),
 (116, 'F_quantity_std_y'),
 (114, 'F_coupon_discount_sum_y'),
 (113, 'F_date_w_nunique_y'),
 (109, 'F_date_w_nunique_x'),
 (108, 'F_selling_price_mean_y'),
 (104, 'F_quantity_max_x'),
 (103, 'F_coupon_discount_min_y'),
 (102, 'F_quantity_mean_x'),
 (102

In [65]:
p = m.predict_proba(Xtest)[:,-1]


[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9


In [67]:
Xtest = df_test.drop(loc, axis=1)


In [68]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier, early_stopping
import pandas as pd
import numpy as np

err = []
y_pred_tot = []
feature_importance_df = pd.DataFrame()

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for i, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = LGBMClassifier(
        n_estimators=5000,
        random_state=42,
        learning_rate=0.03,
        reg_alpha=0.2,
        colsample_bytree=0.5
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='auc',
        callbacks=[early_stopping(stopping_rounds=100)]
    )

    preds = model.predict_proba(X_val, num_iteration=model.best_iteration_)[:, 1]
    print(f"Fold {i} AUC: {roc_auc_score(y_val, preds):.4f}")
    err.append(roc_auc_score(y_val, preds))

    y_pred_tot.append(model.predict_proba(Xtest, num_iteration=model.best_iteration_)[:, 1])

    fold_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': model.feature_importances_,
        'fold': i
    })
    feature_importance_df = pd.concat([feature_importance_df, fold_importance], axis=0)

    y_pred_tot.append(p)

[LightGBM] [Info] Number of positive: 583, number of negative: 62112
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.035322 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13909
[LightGBM] [Info] Number of data points in the train set: 62695, number of used features: 66
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.009299 -> initscore=-4.668507
[LightGBM] [Info] Start training from score -4.668507
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[178]	valid_0's auc: 0.949229	valid_0's binary_logloss: 0.0342881
Fold 1 AUC: 0.9492
[LightGBM] [Info] Number of positive: 583, number of negative: 62112
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.041816 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13903
[LightGBM] [Info] Number of data points in the train set: 6

In [69]:
np.mean(err)

0.9328469521405797

In [71]:
s = pd.DataFrame()
s['id'] = df_test['id']  
s['redemption_status'] = np.mean(y_pred_tot, axis=0)
s.head()

,id,redemption_status
0,3,0.022328
1,4,0.003973
2,5,0.035762
3,8,0.000072
4,10,0.002443


In [72]:
sum(s.redemption_status>0.5)

0

In [73]:
s.to_csv('sfklgbm.csv',index=False)
s.shape

(50226, 2)

### Catboost Model

In [74]:
print(X.shape,Xtest.shape)

X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.25,random_state = 1994,stratify=y)
categorical_features_indices = np.where(X_train.dtypes =='category')[0]
categorical_features_indices

(78369, 72) (50226, 72)


array([3], dtype=int64)

In [75]:
m=CatBoostClassifier(n_estimators=2500,random_state=1994,learning_rate=0.03,eval_metric='AUC')
m.fit(X_train,y_train,eval_set=[(X_val, y_val.values)], early_stopping_rounds=300,verbose=200,cat_features=categorical_features_indices)
p=m.predict_proba(X_val)[:,-1]
print(roc_auc_score(y_val,p))

0:	test: 0.6879207	best: 0.6879207 (0)	total: 204ms	remaining: 8m 30s
200:	test: 0.9076092	best: 0.9076536 (199)	total: 8.81s	remaining: 1m 40s
400:	test: 0.9221216	best: 0.9221386 (399)	total: 17.3s	remaining: 1m 30s
600:	test: 0.9279450	best: 0.9279450 (600)	total: 25.7s	remaining: 1m 21s
800:	test: 0.9314513	best: 0.9315229 (797)	total: 34.2s	remaining: 1m 12s
1000:	test: 0.9309987	best: 0.9315577 (886)	total: 43s	remaining: 1m 4s
1200:	test: 0.9311739	best: 0.9316376 (1036)	total: 51.4s	remaining: 55.6s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.93163755
bestIteration = 1036

Shrink model to first 1037 iterations.
0.9316375500240319


In [76]:
errCB=[]
y_pred_tot_cb=[]
from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=15,shuffle=True,random_state=1994)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m=CatBoostClassifier(n_estimators=5000,random_state=1994,eval_metric='AUC',learning_rate=0.03)
    m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=200,verbose=200,cat_features=categorical_features_indices)
    preds=m.predict_proba(X_test)[:,-1]
    print("err_cb: ",roc_auc_score(y_test,preds))
    errCB.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(Xtest)[:,-1]
    i=i+1
    y_pred_tot_cb.append(p)

0:	test: 0.6244618	test1: 0.6013962	best: 0.6013962 (0)	total: 48.8ms	remaining: 4m 4s
200:	test: 0.9444920	test1: 0.8909311	best: 0.8909311 (200)	total: 11.3s	remaining: 4m 29s
400:	test: 0.9666512	test1: 0.9022845	best: 0.9024304 (398)	total: 21.7s	remaining: 4m 8s
600:	test: 0.9767901	test1: 0.9079819	best: 0.9083880 (571)	total: 32s	remaining: 3m 54s
800:	test: 0.9827597	test1: 0.9126463	best: 0.9128316 (796)	total: 42.4s	remaining: 3m 42s
1000:	test: 0.9872687	test1: 0.9178193	best: 0.9178193 (1000)	total: 53.3s	remaining: 3m 32s
1200:	test: 0.9907984	test1: 0.9179533	best: 0.9181032 (1135)	total: 1m 3s	remaining: 3m 21s
1400:	test: 0.9933203	test1: 0.9200549	best: 0.9204058 (1376)	total: 1m 14s	remaining: 3m 11s
1600:	test: 0.9952484	test1: 0.9221328	best: 0.9222116 (1596)	total: 1m 25s	remaining: 3m
1800:	test: 0.9965825	test1: 0.9232092	best: 0.9232289 (1797)	total: 1m 35s	remaining: 2m 50s
2000:	test: 0.9975666	test1: 0.9234852	best: 0.9236665 (1966)	total: 1m 46s	remaining: 2

1000:	test: 0.9877349	test1: 0.9194479	best: 0.9199511 (941)	total: 55.1s	remaining: 3m 39s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9199510562
bestIteration = 941

Shrink model to first 942 iterations.
err_cb:  0.9199510561566203
0:	test: 0.8090106	test1: 0.8280002	best: 0.8280002 (0)	total: 55.1ms	remaining: 4m 35s
200:	test: 0.9462795	test1: 0.9321025	best: 0.9323480 (193)	total: 11.1s	remaining: 4m 23s
400:	test: 0.9679392	test1: 0.9368963	best: 0.9369566 (389)	total: 21.4s	remaining: 4m 5s
600:	test: 0.9778375	test1: 0.9372947	best: 0.9376006 (572)	total: 32.2s	remaining: 3m 55s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9376006247
bestIteration = 572

Shrink model to first 573 iterations.
err_cb:  0.937600624678001
0:	test: 0.8126877	test1: 0.7820610	best: 0.7820610 (0)	total: 88.8ms	remaining: 7m 24s
200:	test: 0.9451784	test1: 0.9060246	best: 0.9060246 (200)	total: 14s	remaining: 5m 34s
400:	test: 0.9677045	test1: 0.9208044	

In [77]:
np.mean(errCB,0)

0.9352227663347666

In [78]:
s['redemption_status']=np.mean(y_pred_tot_cb,0)
s.head()

,id,redemption_status
0,3,0.013529
1,4,0.004600
2,5,0.057620
3,8,0.000110
4,10,0.001898


In [79]:
sum(s.redemption_status>0.5)

0

In [80]:
s.to_csv('Solfile.csv',index=False)
s.shape

(50226, 2)